In [1]:
from pyspark import SparkConf, __version__ as pyspark_version
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [2]:
main_pyspark_version = ".".join(pyspark_version.split(".")[:-1])
KAFKA_HOST = "localhost:9094"
packages = f"org.apache.spark:spark-sql-kafka-0-10_2.12:{main_pyspark_version}.0"
topic_name = "test"

In [3]:
kafkaParams = {
    "kafka.bootstrap.servers": "localhost:9094",  # Replace with your broker addresses
    "subscribe":f"{topic_name}",  # Replace with your topic name
    "startingOffsets": "earliest",  # or "earliest"
    "key.deserializer": "org.apache.kafka.common.serialization.StringDeserializer",
    "value.deserializer": "org.apache.kafka.common.serialization.StringDeserializer"
}

In [4]:
spark = SparkSession.builder.appName("kafka-spark-example")\
            .config("spark.jars.packages",packages)\
            .config("spark.sql.warehouse.dir","file:///spark-warehouse")

spark.enableHiveSupport().getOrCreate()

In [5]:
df = spark.readStream.format("kafka") \
        .options(**kafkaParams) \
        .load()

AttributeError: 'Builder' object has no attribute 'readStream'

In [ ]:
df.selectExpr("CAST(value AS STRING)")
    
df.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", "false") \
    .foreach(lambda row: print(row)) \
    .start()